In [1]:
import pandas as pd
import os

In [2]:
csv_paths = ['../data/raw/' + file for file in os.listdir('../data/raw/') if file.endswith('csv')] #read paths of all indicator csvs into list
df = None
flag = False

for path in csv_paths: 
    
    indicator_df = pd.read_csv(path,header=2).drop(['Unnamed: 67','Country Code','Indicator Code'],axis=1) #turn each indicator into dataframe  

    indicator_df = pd.melt(indicator_df,id_vars=['Country Name','Indicator Name'],value_name=indicator_df['Indicator Name'][0],var_name='Year').drop('Indicator Name',axis=1) #melt indicator into one column

    if flag:
        df = pd.merge(left=df,right=indicator_df,on=['Country Name','Year'],how='outer') #join with other indicators
    else:
        df = indicator_df
        flag = True

In [3]:
df.head()  #type: ignore

,Country Name,Year,"Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)","Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative)",Poverty headcount ratio at $3.65 a day (2017 PPP) (% of population),"Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative)","Inflation, consumer prices (annual %)",Poverty headcount ratio at $6.85 a day (2017 PPP) (% of population),"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)",Gini index,...,"Population, total",Population ages 65 and above (% of total population),"Cause of death, by non-communicable diseases (% of total)","Unemployment, total (% of total labor force) (modeled ILO estimate)",Poverty gap at $6.85 a day (2017 PPP) (%),"Hospital beds (per 1,000 people)","Life expectancy at birth, total (years)","Physicians (per 1,000 people)","Educational attainment, at least completed post-secondary, population 25+, total (%) (cumulative)","Educational attainment, at least completed primary, population 25+ years, total (%) (cumulative)"
0,Aruba,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54608.0,2.373279,NaN,NaN,NaN,NaN,64.152000,NaN,NaN,NaN
1,Africa Eastern and Southern,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,130692579.0,2.972568,NaN,NaN,NaN,1.966442,44.085552,NaN,NaN,NaN
2,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8622466.0,2.833029,NaN,NaN,NaN,0.170627,32.535000,0.035,NaN,NaN
3,Africa Western and Central,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,97256290.0,3.301681,NaN,NaN,NaN,0.786057,37.845152,NaN,NaN,NaN
4,Angola,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5357195.0,3.080044,NaN,NaN,NaN,2.061462,38.211000,0.067,NaN,NaN


In [4]:
feature_names = ['Year',
 'Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)',
 'Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative)',
 'Inflation, consumer prices (annual %)',
 'GDP (current US$)',
 'Cause of death, by injury (% of total)',
 "Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative)",
 'Educational attainment, at least completed short-cycle tertiary, population 25+, total (%) (cumulative)',
 'Population, total',
 'Unemployment, total (% of total labor force) (modeled ILO estimate)',
 'Hospital beds (per 1,000 people)',
 'Life expectancy at birth, total (years)',
 'Physicians (per 1,000 people)',
 'Country Name',
 'Gini index'
]

for col in df.columns: #type:ignore
    if col.find('Poverty') > -1:
        feature_names.append(col)

df = df[feature_names] #type:ignore

In [5]:
df.head()

,Year,"Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)","Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative)","Inflation, consumer prices (annual %)",GDP (current US$),"Cause of death, by injury (% of total)","Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative)","Educational attainment, at least completed short-cycle tertiary, population 25+, total (%) (cumulative)","Population, total","Unemployment, total (% of total labor force) (modeled ILO estimate)",...,"Life expectancy at birth, total (years)","Physicians (per 1,000 people)",Country Name,Gini index,Poverty headcount ratio at $3.65 a day (2017 PPP) (% of population),Poverty headcount ratio at $6.85 a day (2017 PPP) (% of population),Poverty gap at $2.15 a day (2017 PPP) (%),Poverty gap at $3.65 a day (2017 PPP) (%),Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population),Poverty gap at $6.85 a day (2017 PPP) (%)
0,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54608.0,NaN,...,64.152000,NaN,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1960,NaN,NaN,NaN,2.112502e+10,NaN,NaN,NaN,130692579.0,NaN,...,44.085552,NaN,Africa Eastern and Southern,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1960,NaN,NaN,NaN,5.377778e+08,NaN,NaN,NaN,8622466.0,NaN,...,32.535000,0.035,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1960,NaN,NaN,NaN,1.044764e+10,NaN,NaN,NaN,97256290.0,NaN,...,37.845152,NaN,Africa Western and Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5357195.0,NaN,...,38.211000,0.067,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df['Country Name'] = df['Country Name'].astype('category')   #change country to category
df['Year'] =  df['Year'].astype('Int64')   #change year to int

In [7]:
from copy import deepcopy

gini = deepcopy(df[ ~df['Gini index'].isna()])

# percentage of missing values for each column
pd.Series([gini[col].isna().sum()/len(gini) for col in gini.columns],index=gini.columns).sort_values(ascending=False) 

Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative)                     0.888946
Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative)            0.863120
Cause of death, by injury (% of total)                                                                     0.854339
Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)         0.681818
Educational attainment, at least completed short-cycle tertiary, population 25+, total (%) (cumulative)    0.675620
Hospital beds (per 1,000 people)                                                                           0.356405
Physicians (per 1,000 people)                                                                              0.331612
Unemployment, total (% of total labor force) (modeled ILO estimate)                                        0.087293
Inflation, consumer prices (annual %)                                   

In [8]:
len(gini)

1936

In [9]:
def interpolate(radius):
    new_df = deepcopy(gini)

    for col in new_df.columns:
        if new_df[col].isna().sum() > 0 and not col in ['Country Name','Year']:
            
            to_impute = new_df[new_df[col].isna()]
            for i in range(len(to_impute)):
                country = to_impute['Country Name'].iloc[i]
                year = to_impute['Year'].iloc[i]
                surrounding = df[(df['Country Name'] == country) & ( abs(df['Year'] - year) <= radius)][col]
                new_df.loc[(new_df['Year'] == year) & (new_df['Country Name'] == country),col] = surrounding.mean()
                        
    print(f'radius: {radius}')                        
    return new_df

In [10]:
interpol_meta = {
    'radius':[],
    'size':[],
    'countries':[],
    'yearFrom':[],
    'yearTo':[]
}

for rad in range(1,6):
    rad_df = interpolate(rad).dropna()
    #rad_df.to_csv(f'../data/interim/radius{rad}.csv')
    interpol_meta['radius'].append(rad)
    interpol_meta['size'].append(len(rad_df))
    interpol_meta['countries'].append( len(rad_df['Country Name'].unique()) )
    interpol_meta['yearFrom'].append(rad_df['Year'].min())
    interpol_meta['yearTo'].append(rad_df['Year'].max())

radius: 1
radius: 2
radius: 3
radius: 4
radius: 5


In [11]:
pd.DataFrame(interpol_meta)

,radius,size,countries,yearFrom,yearTo
0,1,156,48,2009,2020
1,2,339,60,2002,2021
2,3,429,67,2002,2021
3,4,512,72,2002,2021
4,5,592,74,1996,2022


In [12]:
sample = interpolate(3).dropna()

sample.head()

radius: 3


,Year,"Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)","Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative)","Inflation, consumer prices (annual %)",GDP (current US$),"Cause of death, by injury (% of total)","Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative)","Educational attainment, at least completed short-cycle tertiary, population 25+, total (%) (cumulative)","Population, total","Unemployment, total (% of total labor force) (modeled ILO estimate)",...,"Life expectancy at birth, total (years)","Physicians (per 1,000 people)",Country Name,Gini index,Poverty headcount ratio at $3.65 a day (2017 PPP) (% of population),Poverty headcount ratio at $6.85 a day (2017 PPP) (% of population),Poverty gap at $2.15 a day (2017 PPP) (%),Poverty gap at $3.65 a day (2017 PPP) (%),Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population),Poverty gap at $6.85 a day (2017 PPP) (%)
11420,2002,88.543221,0.00000,0.757421,4.395616e+10,8.507397,16.28310,38.141708,48202470.0,10.140,...,68.275610,3.0188,Ukraine,29.0,17.0,61.9,0.5,3.7,2.7,20.8
11686,2003,88.543221,0.00000,5.179678,5.201035e+10,8.507397,16.28310,38.141708,47812949.0,9.060,...,68.210732,3.0096,Ukraine,28.7,10.4,54.1,0.2,2.1,1.1,16.0
12624,2007,95.692841,0.03193,10.230103,3.802571e+09,10.150055,15.44759,17.622650,5268400.0,2.677,...,67.895122,2.3580,Kyrgyz Republic,33.9,33.6,74.2,1.7,9.5,8.8,31.8
12687,2007,61.490292,0.25529,4.168633,2.129598e+10,12.359977,2.37209,21.358910,3431614.0,5.844,...,76.154000,1.3032,Panama,52.7,14.9,30.5,2.2,5.7,6.7,13.6
12879,2008,80.522840,0.79249,4.060305,2.754475e+11,5.055017,9.20796,30.182824,4489544.0,6.770,...,80.095122,4.0144,Ireland,30.9,0.3,0.8,0.1,0.2,0.2,0.4


In [13]:
new_names = {
    'Inflation, consumer prices (annual %)': 'Inflation',
    'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)':'COD, by communicable diseases',
    'Cause of death, by injury (% of total)':'COD, by injury',
    'Cause of death, by non-communicable diseases (% of total)':'COD, by non-communicable diseases',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)':'Unemployment',
    'Life expectancy at birth, total (years)':'Life expectancy',
}

for col in sample.columns:
    start = col.find('Educational')
    stop = col.find(', population')
    if start > -1:
        new_names[col] = 'EA, '+ col[start+24:stop]

    stop = col.find(' a day')
    if stop > -1:
        new_names[col] = col[0:stop]

sample.rename(columns=new_names,inplace=True)

In [14]:
sample['Country Name'].unique().tolist() #list of countries kept

['Ukraine',
 'Kyrgyz Republic',
 'Panama',
 'Ireland',
 'Korea, Rep.',
 'Mexico',
 'Bolivia',
 'Georgia',
 'Iran, Islamic Rep.',
 'Paraguay',
 'Slovak Republic',
 'Belgium',
 'Bangladesh',
 'United Kingdom',
 'Hungary',
 'Israel',
 'Moldova',
 'Malta',
 'Netherlands',
 'Pakistan',
 'Poland',
 'Rwanda',
 'El Salvador',
 'Sweden',
 'Turkiye',
 'Uruguay',
 'Austria',
 'Switzerland',
 'Cyprus',
 'Czechia',
 'Germany',
 'France',
 'Greece',
 'Lithuania',
 'Latvia',
 'Niger',
 'Norway',
 'Portugal',
 'Romania',
 'Senegal',
 'Slovenia',
 'Albania',
 'Bhutan',
 'Denmark',
 'Indonesia',
 'Serbia',
 'United Arab Emirates',
 'Brazil',
 'Dominican Republic',
 'Spain',
 'Malaysia',
 'Australia',
 'Guatemala',
 'Mozambique',
 'Costa Rica',
 'Egypt, Arab Rep.',
 'Philippines',
 'Tajikistan',
 'United States',
 'Belarus',
 'Iceland',
 'Vietnam',
 'Estonia',
 'Italy',
 'North Macedonia',
 'Myanmar',
 'Mongolia']

In [15]:
sample['GDP per capita'] = sample['GDP (current US$)']/sample['Population, total']

sample.to_csv('../data/interim/radius3_selected_features.csv')

In [16]:
sample.head()

,Year,"EA, at least completed lower secondary","EA, Doctoral or equivalent",Inflation,GDP (current US$),"COD, by injury","EA, at least Master's or equivalent","EA, at least completed short-cycle tertiary","Population, total",Unemployment,...,"Physicians (per 1,000 people)",Country Name,Gini index,Poverty headcount ratio at $3.65,Poverty headcount ratio at $6.85,Poverty gap at $2.15,Poverty gap at $3.65,Poverty headcount ratio at $2.15,Poverty gap at $6.85,GDP per capita
11420,2002,88.543221,0.00000,0.757421,4.395616e+10,8.507397,16.28310,38.141708,48202470.0,10.140,...,3.0188,Ukraine,29.0,17.0,61.9,0.5,3.7,2.7,20.8,911.906871
11686,2003,88.543221,0.00000,5.179678,5.201035e+10,8.507397,16.28310,38.141708,47812949.0,9.060,...,3.0096,Ukraine,28.7,10.4,54.1,0.2,2.1,1.1,16.0,1087.788007
12624,2007,95.692841,0.03193,10.230103,3.802571e+09,10.150055,15.44759,17.622650,5268400.0,2.677,...,2.3580,Kyrgyz Republic,33.9,33.6,74.2,1.7,9.5,8.8,31.8,721.769526
12687,2007,61.490292,0.25529,4.168633,2.129598e+10,12.359977,2.37209,21.358910,3431614.0,5.844,...,1.3032,Panama,52.7,14.9,30.5,2.2,5.7,6.7,13.6,6205.821576
12879,2008,80.522840,0.79249,4.060305,2.754475e+11,5.055017,9.20796,30.182824,4489544.0,6.770,...,4.0144,Ireland,30.9,0.3,0.8,0.1,0.2,0.2,0.4,61353.106563
